In [1]:
import bs4 as bs
import pickle # remove pickle and sub in csv
import requests


html = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(html.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

print(html) #error handling, 200 = good, 404 bad etc

<Response [200]>


In [2]:
import datetime as dt
import os
import pandas_datareader.data as pdr


def save_sp500_tickers():
    """A function to save sp500 data as pickle"""
    # TODO modify to change to csv
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        ticker = ticker[:-1]
        tickers.append(ticker)

    with open("sptickers.pickle", "wb") as f: # TODO remove pickle swap csv
        pickle.dump(tickers, f)
    return tickers


def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle", "rb") as f:
            tickers = pickle.load(f)
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')

    start = dt.datetime(2010, 1, 1)
    end = dt.datetime.now()

    for ticker in tickers:
        if not os.path.exists('yahoo_stock_dfs/{}.csv'.format(ticker)):
            df = pdr.DataReader(ticker.replace('.', '-'), 'yahoo', start, end)
            df.reset_index(inplace=True)
            df.set_index("Date", inplace=True)
            df.to_csv('yahoo_stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))

CSV_SAVE_PATH = '/Users/wadewilson/Library/Mobile Documents/com~apple~CloudDocs/Data/Financial/yahoo_sp500/'